In [30]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [31]:
#step1.import required lib
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

In [32]:
#step2.get url from amozon
URL="https://www.amazon.in/s?k=laptops&i=computers&crid=PGSZ07K9FJJD&sprefix=laptops%2Ccomputers%2C413&ref=nb_sb_noss_2"

In [33]:
#make header to mimic a browser visit
headers = {
   "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36"
}

In [34]:
#create the empty list to store the data
data=[]
print("Data list cleared ")

Data list cleared 


In [35]:
#scrap the data from the wedsite
response=requests.get(URL,headers=headers)
response.status_code

200

In [36]:
# beautifulsoup to parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")
soup

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-in"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link crossorigin="" href="https://images-eu.ssl-images-amazon.com" rel="preconnect"/>
<link crossorigin="" href="https://m.media-amazon.com" rel="preconnect"/>
<!-- sp:end-feature:cs-optimization -->
<!-- sp:feature:csm:head-open-part2 -->
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.st

In [37]:
for page in range(1, 2):  
    params = {"k": "laptop", "page": page}
    
    # Use verify=False to bypass SSL verification (for testing purposes)
    response = requests.get(URL, params=params, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    
    products = soup.find_all("div", {"data-component-type": "s-search-result"})
    
    for product in products:
        # Extract the product title
        title_tag = product.find("h2")
        if not title_tag:
            continue
        title_text = title_tag.get_text(strip=True)

        
        # Extract the product price
        price_tag = product.find("span", {"class": "a-price-whole"})
        price_text = price_tag.get_text(strip=True) if price_tag else "N/A"
        
        # Extract the product brand
        match = re.match(r'^\W*([A-Za-z]+)', title_text)
        brand = match.group(1).upper() if match else "UNKNOWN"

        # Extract RAM (improved version)
        ram_match = re.search(r'(\d+)\s*GB\s*(RAM|DDR\d+|LPDDR\d+)', title_text, re.IGNORECASE)
        ram = ram_match.group(1) + "GB" if ram_match else "N/A"
        
        # Extract the product rating 
        rating_tag = product.find("span", class_="a-icon-alt")
        rating = "N/A"
        if rating_tag:
            rating_match = re.search(r'(\d+\.?\d*)', rating_tag.get_text())
            rating = rating_match.group(1) if rating_match else "N/A"

        # Extract the Product SSD - Simplified
        ssd_match = re.search(r'(\d+)\s*(GB|TB)\s*(?:SSD|NVMe)', title_text, re.IGNORECASE) or re.search(r'(\d+)\s*(GB|TB)', title_text)
        ssd = (ssd_match.group(1) + ssd_match.group(2).upper()) if ssd_match else "N/A"
      

        # Extract the Product Windows Version - Improved version
        windows_version = "N/A"
        
        # Try: "Windows 11", "Windows 10", "Win11", "Win10", etc.
        windows_match = re.search(r'(?:Windows\s*|Win\s*)(\d+)', title_text, re.IGNORECASE)
        if windows_match:
            windows_version = "Windows " + windows_match.group(1)
        
        # Extract the Product Color
        color_match = re.search(r'\b(Black|Silver|Gray|Grey|White|Blue|Red|Gold|Green|Brown|Pink|Purple|Yellow|Orange|Champagne|Midnight|Space|Cosmic|Stardust|Graphite|Ash|Onyx|Platinum|Metallic)\b', title_text, re.IGNORECASE)
        color = color_match.group(1) if color_match else "N/A"

        # Extract the Processor (Intel, AMD, Apple M, etc)
        processor = "N/A"
        
        # Try Intel processors - more flexible patterns
        intel_match = re.search(r'Intel\s+(?:Core\s+)?(?:i[3579]|m\d|Pentium|Celeron|Atom|Xeon)[-\w]*', title_text, re.IGNORECASE)
        if intel_match:
            processor = intel_match.group(0).strip()
        
        # Try AMD Ryzen/Athlon processors
        elif re.search(r'AMD', title_text, re.IGNORECASE):
            amd_match = re.search(r'AMD\s+(?:Ryzen|Athlon)[\s\w]*', title_text, re.IGNORECASE)
            if amd_match:
                processor = amd_match.group(0).strip()
        
        # Try Apple M series
        elif re.search(r'Apple\s+M', title_text, re.IGNORECASE):
            apple_match = re.search(r'Apple\s+M\d+\w*', title_text, re.IGNORECASE)
            if apple_match:
                processor = apple_match.group(0).strip()
        
        # Try other processors
        else:
            other_match = re.search(r'Qualcomm\s+Snapdragon|MediaTek|ARM|Exynos', title_text, re.IGNORECASE)
            if other_match:
                processor = other_match.group(0).strip()


        # Store the extracted data in a dictionary and append to the list
        data.append({
            "Title": title_text,
            "Price": price_text,
            "Brand": brand,
            'RAM': ram,
            "Rating": rating,
            "Storage": ssd,
            "Windows": windows_version,
            "Color": color,
            "Processor": processor
        })
    print(f"Page {page} scraped")
    time.sleep(1)


Page 1 scraped


In [38]:
for product in data:
    print("Title:", product["Title"])
    print("Price:", product["Price"])
    print("Brand:", product["Brand"])
    print("RAM:", product.get("RAM"))
    print("Processor:", product.get("Processor"))
    print("Rating:", product.get("Rating"))
    print("Storage:", product.get("Storage"))
    print("Windows:", product.get("Windows"))
    print("Color:", product.get("Color"))
    print("-" * 50)


Title: HP Omnibook 5 OLED (Previously Pavilion), Snapdragon X Processor (16GB LPDDR5x,1TB SSD) 2K OLED,16''/40.6cm, Win11, M365*Office24, Glacier Silver, 1.59kg, fb0001QU, Backlit, Next-Gen AI Laptop
Price: 69,990
Brand: HP
RAM: 16GB
Processor: N/A
Rating: 4.1
Storage: 1TB
Windows: Windows 11
Color: Silver
--------------------------------------------------
Title: Dell 15, Intel Core i5 13th Gen-1334U Processor, 16GB DDR4, 512GB SSD, FHD, 15.6"/39.62cm, Windows 11, MSO 2024, Grey, 1.66kg, [Vostro 3530], 120Hz 250 nits, 15 Month McAfee, Thin & Light, Laptop
Price: 56,990
Brand: DELL
RAM: 16GB
Processor: Intel Core i5
Rating: 3.7
Storage: 512GB
Windows: Windows 11
Color: Grey
--------------------------------------------------
Title: Dell 14, AMD Ryzen AI R5-340, 16GB LPDDR5X, 512GB SSD, FHD+, 14"/35.56cm, Win11, MSO 2024, Ice Blue, 1.52 Kg, [Dell 14], 300 Nits IPS, AMD Radeon Graphics, Backlit+FPS Keyboard, AI Powered Laptop
Price: 73,490
Brand: DELL
RAM: 16GB
Processor: AMD Ryzen AI R5
R

In [39]:
# Create a DataFrame from the data list
df = pd.DataFrame(data)
df.head()

,Title,Price,Brand,RAM,Rating,Storage,Windows,Color,Processor
0,"HP Omnibook 5 OLED (Previously Pavilion), Snap...","69,990",HP,16GB,4.1,1TB,Windows 11,Silver,N/A
1,"Dell 15, Intel Core i5 13th Gen-1334U Processo...","56,990",DELL,16GB,3.7,512GB,Windows 11,Grey,Intel Core i5
2,"Dell 14, AMD Ryzen AI R5-340, 16GB LPDDR5X, 51...","73,490",DELL,16GB,3.7,512GB,Windows 11,Blue,AMD Ryzen AI R5
3,"EBook 11.6"" HD Laptop | Best Student & Office ...","10,990",EBOOK,4GB,3.8,4GB,Windows 11,Black,N/A
4,JioBook 11 with Lifetime Office | Android 4G L...,"10,990",JIOBOOK,4GB,2.9,4GB,N/A,Blue,Mediatek


In [40]:
print("First 3 product titles:")
for i in range(min(3, len(data))):
    print(f"\n{i+1}. Title: {data[i]['Title']}")
    print(f"   Processor: {data[i]['Processor']}")


First 3 product titles:

1. Title: HP Omnibook 5 OLED (Previously Pavilion), Snapdragon X Processor (16GB LPDDR5x,1TB SSD) 2K OLED,16''/40.6cm, Win11, M365*Office24, Glacier Silver, 1.59kg, fb0001QU, Backlit, Next-Gen AI Laptop
   Processor: N/A

2. Title: Dell 15, Intel Core i5 13th Gen-1334U Processor, 16GB DDR4, 512GB SSD, FHD, 15.6"/39.62cm, Windows 11, MSO 2024, Grey, 1.66kg, [Vostro 3530], 120Hz 250 nits, 15 Month McAfee, Thin & Light, Laptop
   Processor: Intel Core i5

3. Title: Dell 14, AMD Ryzen AI R5-340, 16GB LPDDR5X, 512GB SSD, FHD+, 14"/35.56cm, Win11, MSO 2024, Ice Blue, 1.52 Kg, [Dell 14], 300 Nits IPS, AMD Radeon Graphics, Backlit+FPS Keyboard, AI Powered Laptop
   Processor: AMD Ryzen AI R5


In [59]:
#STEP:1 WEB SCRAPING (Amazon Laptop Data)
all_data = []
for page in range(1, 6):   # 5 pages ≈ 100+ products
    print(f"Scraping page {page}...")
    response = requests.get(base_url.format(page), headers=headers)
    soup = BeautifulSoup(response.content, "lxml")
    products = soup.find_all("div", {"data-component-type": "s-search-result"})
    for product in products:
        title_tag = product.h2
        title = title_tag.text.strip() if title_tag else None
        if not title:
            continue
        page_text = product.text
        # Brand
        brand = re.match(r"^\w+", title).group() if title else None
        # Price
        price = re.search(r"₹\s?([\d,]+)", page_text)
        price = price.group(1).replace(",", "") if price else None
        # Rating
        rating = re.search(r"(\d+(\.\d+)?)\s?out of 5", page_text)
        rating = rating.group(1) if rating else None
        # Reviews count
        reviews = re.search(r"([\d,]+)\s?ratings", page_text)
        reviews = reviews.group(1).replace(",", "") if reviews else None
        # Processor
        processor = re.search(r"(i[3579]|Ryzen\s?\d)", title)
        processor = processor.group() if processor else None
        # RAM
        ram = re.search(r"(\d+)\s?GB", title)
        ram = ram.group(1)+"GB" if ram else None
        # Storage
        storage = re.search(r"(\d+)\s?(GB|TB)\s?(SSD|HDD)?", title)
        storage = storage.group() if storage else None
        # Screen Size
        screen = re.search(r"(\d+(\.\d+)?)\s?(Inch|Inches)", title)
        screen = screen.group(1) if screen else None
        # Windows
        windows_match = re.search(r'(?:Windows\s*|Win\s*)(\d+)', title, re.IGNORECASE)
        windows_version = "Windows " + windows_match.group(1) if windows_match else None
        all_data.append([
            title, brand, price, rating, reviews,
            processor, ram, storage, screen, windows_version
        ])
time.sleep(2)
columns = [
    "product_title", "brand", "price", "rating", "reviews",
    "processor", "RAM", "storage", "screen_size", "windows"
]
df = pd.DataFrame(all_data, columns=columns)
# Save Raw File
df.to_csv("amazon_laptop_raw.csv", index=False)
print("Raw file saved successfully!")

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Raw file saved successfully!


In [60]:
#Step 2: DATA CLEANING & DATAFRAME OPERATIONS python
df = pd.read_csv("amazon_laptop_raw.csv")
print("Shape:", df.shape)
print(df.info())
print(df.isnull().sum())
# Remove duplicates
df.drop_duplicates(inplace=True)
# Convert numeric columns
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["reviews"] = pd.to_numeric(df["reviews"], errors="coerce")
# Drop rows without price
df = df.dropna(subset=["price"])
# Save Cleaned File
df.to_csv("amazon_laptop_cleaned.csv", index=False)
print("Cleaned file saved successfully!")

Shape: (0, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_title  0 non-null      object
 1   brand          0 non-null      object
 2   price          0 non-null      object
 3   rating         0 non-null      object
 4   reviews        0 non-null      object
 5   processor      0 non-null      object
 6   RAM            0 non-null      object
 7   storage        0 non-null      object
 8   screen_size    0 non-null      object
 9   windows        0 non-null      object
dtypes: object(10)
memory usage: 132.0+ bytes
None
product_title    0
brand            0
price            0
rating           0
reviews          0
processor        0
RAM              0
storage          0
screen_size      0
windows          0
dtype: int64
Cleaned file saved successfully!


In [57]:
#step 3: ANALYSIS QUESTIONS
df = pd.read_csv("amazon_laptop_cleaned.csv")
# 1️. Brand with highest average price
print("\n1. Brand with Highest Average Price:")
print(df.groupby("brand")["price"].mean().sort_values(ascending=False).head(1))
# 2️. Brand with highest average rating
print("\n2. Brand with Highest Average Rating:")
print(df.groupby("brand")["rating"].mean().sort_values(ascending=False).head(1))
# 3️. Top 5 most reviewed laptops
print("\n3. Top 5 Most Reviewed Laptops:")
print(df.sort_values("reviews", ascending=False)[["product_title","reviews"]].head())
# 4️. Average discount percentage (if discount column exists)
if "discount" in df.columns:
    print("\n4. Average Discount %:")
    print(df["discount"].mean())
else:
    print("\n4. Discount column not available")
# 5️. Price vs Rating correlation
print("\n5. Price vs Rating Correlation:")
print(df["price"].corr(df["rating"]))
# 6️. Distribution by RAM
print("\n6. Distribution by RAM:")
print(df["RAM"].value_counts())
# 7️. Most common screen size
print("\n7. Most Common Screen Size:")
print(df["screen_size"].value_counts().head(1))
# 8️. Laptops under ₹50,000
print("\n8. Laptops under ₹50,000:")
print(len(df[df["price"] < 50000]))


1. Brand with Highest Average Price:
Series([], Name: price, dtype: object)

2. Brand with Highest Average Rating:
Series([], Name: rating, dtype: object)

3. Top 5 Most Reviewed Laptops:
Empty DataFrame
Columns: [product_title, reviews]
Index: []

4. Discount column not available

5. Price vs Rating Correlation:
nan

6. Distribution by RAM:
Series([], Name: count, dtype: int64)

7. Most Common Screen Size:
Series([], Name: count, dtype: int64)

8. Laptops under ₹50,000:
0
